From [naturalscenesdataset.org](https://naturalscenesdataset.org):
```text
The Natural Scenes Dataset (NSD) is a large-scale fMRI dataset conducted at ultra-high-field (7T) strength at the Center of Magnetic Resonance Research (CMRR) at the University of Minnesota. The dataset consists of whole-brain, high-resolution (1.8-mm isotropic, 1.6-s sampling rate) fMRI measurements of 8 healthy adult subjects while they viewed thousands of color natural scenes over the course of 30–40 scan sessions. While viewing these images, subjects were engaged in a continuous recognition task in which they reported whether they had seen each given image at any point in the experiment. These data constitute a massive benchmark dataset for computational models of visual representation and cognition, and can support a wide range of scientific inquiry.
```


In [1]:
from cloudpathlib import S3Path, S3Client
from pathlib import Path

# Set up our cache path:
cache_path = Path('/tmp/cache')
if not cache_path.exists():
    cache_path.mkdir()

# Create the root S3Path for the NSD:
nsd_base_path = S3Path(
    's3://natural-scenes-dataset/',
    client=S3Client(
        no_sign_request=True,
        local_cache_dir=cache_path))

In [2]:
import os
import os.path as op
import glob
import nibabel as nib
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from nilearn import plotting

import urllib.request, zipfile
try:
    from pycocotools.coco import COCO
except ImportError as e:
    !pip install pycocotools
    from pycocotools.coco import COCO


#from nsd_access import NSDAccess

%matplotlib inline

  Obtaining dependency information for pycocotools from https://files.pythonhosted.org/packages/ba/64/0451cf41a00fd5ac4501de4ea0e395b7d909e09d665e56890b5d3809ae26/pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
Using cached pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (426 kB)


In [3]:
from utils import ls, crawl

nsd_pppath = nsd_base_path / 'nsddata_betas' / 'ppdata'

ls(nsd_pppath / 'subj01' / 'fsaverage' / 'betas_fithrf_GLMdenoise_RR')

[S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session01.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session02.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session03.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session04.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session05.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session06.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session07.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh

### Loading data in volume space (1.8mm isotropic resolution)

In [4]:
vimg_filename = nsd_pppath / 'subj01' / 'func1pt8mm' / 'betas_fithrf_GLMdenoise_RR' / 'betas_session01.nii.gz'
vimg = nib.load(vimg_filename.fspath)
vimg = vimg.dataobj

# reshape to vector of nvoxels x ntimepoints
vimg = vimg.reshape([np.prod(vimg.shape[0:-1]),vimg.shape[-1]])

vimg.shape

[699192, 750]

### Loading data in surface space (fsaverage)

In [5]:
simg_filename = nsd_pppath / 'subj01' / 'fsaverage' / 'betas_fithrf_GLMdenoise_RR' / 'lh.betas_session01.mgh'
simg = nib.load(simg_filename.fspath)
simg = simg.dataobj

# reshape to vector of nvoxels x ntimepoints
simg = simg.reshape([np.prod(simg.shape[0:-1]),simg.shape[-1]])

simg.shape

[163842, 750]

### Loading in stimuli data

In [6]:
ls(nsd_base_path)
nsd_stimuli = nsd_base_path / 'nsddata_stimuli' / 'stimuli' / 'nsd'
ls(nsd_stimuli)

[S3Path('s3://natural-scenes-dataset/nsddata_stimuli/stimuli/nsd/nsd_stimuli.hdf5')]

In [7]:
f1 = h5py.File((nsd_stimuli / 'nsd_stimuli.hdf5').fspath, 'r+')
print("Keys: %s" % f1.keys())
dset = f1['imgBrick']
#f1.close

Keys: <KeysViewHDF5 ['imgBrick']>


In [8]:
dset

<HDF5 dataset "imgBrick": shape (73000, 425, 425, 3), type "|u1">

In [9]:
dset[0].shape

(425, 425, 3)

In [10]:
#73000 images, 425x425 shape and 3 RGB color dimensions

### Load in stimulus set for subj01

In [27]:
stimdata_s1 = nsd_base_path / 'nsddata' / 'ppdata' / 'subj01' / 'behav' / 'responses.tsv'
#ls(stimdata_s1)
expdata = pd.read_table(stimdata_s1)
expdata

,SUBJECT,SESSION,RUN,TRIAL,73KID,10KID,TIME,ISOLD,ISCORRECT,RT,CHANGEMIND,MEMORYRECENT,MEMORYFIRST,ISOLDCURRENT,ISCORRECTCURRENT,TOTAL1,TOTAL2,BUTTON,MISSINGDATA
0,1,1,1,1,46003,626,0.505082,0,1.0,803.529781,0.0,NaN,NaN,0,1.0,1,0,1.0,0
1,1,1,1,2,61883,5013,0.505128,0,1.0,972.261383,0.0,NaN,NaN,0,1.0,1,0,1.0,0
2,1,1,1,3,829,4850,0.505175,0,1.0,742.351236,0.0,NaN,NaN,0,1.0,1,0,1.0,0
3,1,1,1,4,67574,8823,0.505221,0,1.0,747.518479,0.0,NaN,NaN,0,1.0,1,0,1.0,0
4,1,1,1,5,16021,9538,0.505267,0,1.0,547.422774,0.0,NaN,NaN,0,1.0,1,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,40,12,58,13774,8984,262.629551,1,0.0,1275.300175,0.0,20963.0,21540.0,0,1.0,1,0,1.0,0
29996,1,40,12,59,66768,6026,262.629597,1,1.0,661.379768,0.0,16.0,17622.0,1,1.0,0,1,2.0,0
29997,1,40,12,60,53168,4841,262.629644,1,1.0,786.811781,0.0,9483.0,11912.0,0,0.0,0,1,2.0,0
29998,1,40,12,61,1944,7323,262.629690,1,1.0,502.626801,0.0,83.0,12162.0,1,1.0,0,1,2.0,0


In [12]:
sub_df = expdata[["SUBJECT","SESSION","RUN","TRIAL","73KID"]]
sub_df

,SUBJECT,SESSION,RUN,TRIAL,73KID
0,1,1,1,1,46003
1,1,1,1,2,61883
2,1,1,1,3,829
3,1,1,1,4,67574
4,1,1,1,5,16021
...,...,...,...,...,...
29995,1,40,12,58,13774
29996,1,40,12,59,66768
29997,1,40,12,60,53168
29998,1,40,12,61,1944


In [13]:
[min(sub_df['73KID']), max(sub_df['73KID'])]

[14, 73000]

### Read in the annotated labels

In [14]:
#Find cocoIDs for each stimulus
ls(nsd_base_path)
nsd_cocoIDs = nsd_base_path / 'nsddata' / 'experiments' / 'nsd' / 'nsd_stim_info_merged.csv'

In [15]:
cocoID_DF = pd.read_csv(nsd_cocoIDs)
cocoID = cocoID_DF['cocoId']
cocoID

0        532481
1        245764
2        385029
3        311303
4        393226
          ...  
72995    518071
72996    255930
72997    255934
72998    518080
72999    518083
Name: cocoId, Length: 73000, dtype: int64

In [16]:
#add respective cocoID to the dataframe above to match the 73KID

In [17]:
sub_cocoID = []
for trial in sub_df["73KID"]:
    sub_cocoID.append(cocoID.iloc[trial-1])

In [166]:
sub_cocoid = pd.DataFrame({"COCOID": sub_cocoID})
sub_DF = pd.merge(sub_df, sub_cocoid, left_index=True, right_index=True)
sub_DF
#len(pd.unique(sub_DF['COCOID']))

,SUBJECT,SESSION,RUN,TRIAL,73KID,COCOID
0,1,1,1,1,46003,412922
1,1,1,1,2,61883,474858
2,1,1,1,3,829,320696
3,1,1,1,4,67574,234676
4,1,1,1,5,16021,301595
...,...,...,...,...,...,...
29995,1,40,12,58,13774,32606
29996,1,40,12,59,66768,388123
29997,1,40,12,60,53168,179070
29998,1,40,12,61,1944,13597


In [19]:
#read the labels of the cocoIDs and add column to the data frame

In [86]:
import urllib.request
import zipfile
import json
urllib.request.urlretrieve('http://images.cocodataset.org/annotations/annotations_trainval2017.zip', 'coco_annotations_2017.zip')

with zipfile.ZipFile('coco_annotations_2017.zip', 'r') as zip_ref:
    zip_ref.extractall()

with open('./annotations/instances_train2017.json') as json_data:
  file_contents = json_data.read()

data = json.loads(file_contents)
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [170]:
#urllib.request.urlretrieve('http://images.cocodataset.org/annotations/image_info_unlabeled2017.zip', 'image_info_unlabeled2017.zip')

#with zipfile.ZipFile('image_info_unlabeled2017.zip', 'r') as zip_ref:
#    zip_ref.extractall()

#with open('./annotations/image_info_unlabeled2017.json') as json_data:
#  file_contents = json_data.read()

#data_unlab = json.loads(file_contents)
#data_unlab.keys()

In [21]:
annotations = data['annotations']
categories = data['categories']

In [71]:
ids_list = [data['annotations'][i]['id'] for i in range(len(data['annotations']))]
coco_image_ids_list = [data['annotations'][i]['image_id'] for i in range(len(data['annotations']))]
#[i for i, x in enumerate(np.array(coco_image_ids_list)==320696) if x]
len(pd.unique(coco_image_ids_list))
min(coco_image_ids_list)

9

In [73]:
#data['annotations']

In [81]:
image_id_to_text = {data['categories'][i]['id'] : data['categories'][i]['name'] for i in range(80)}
cat_id_to_text = {data['categories'][i]['id'] : data['categories'][i]['supercategory'] for i in range(80)}
#cat_id_to_text

In [157]:
image_labels_dict_numeric = {image_id : [] for image_id in coco_image_ids_list}
image_labels_dict_text = {image_id : [] for image_id in coco_image_ids_list}
cat_labels_dict_numeric = {supercategory : [] for supercategory in coco_image_ids_list}
cat_labels_dict_text = {supercategory : [] for supercategory in coco_image_ids_list}

for item in data['annotations']:
    image_labels_dict_numeric[item['image_id']].append(item['category_id'])
    image_labels_dict_text[item['image_id']].append(image_id_to_text[item['category_id']])
    cat_labels_dict_numeric[item['image_id']].append(item['category_id'])
    cat_labels_dict_text[item['image_id']].append(cat_id_to_text[item['category_id']])
    
#cat_labels_dict_text

In [158]:
for dictionary in [image_labels_dict_numeric, image_labels_dict_text]:
    for key,value in dictionary.items():
        value = list(set(value))
        value.sort()
        dictionary[key] = value

In [168]:
for dictionary in [cat_labels_dict_numeric, cat_labels_dict_text]:
    for key,value in dictionary.items():
        value = list(set(value))
        value.sort()
        dictionary[key] = value
        
#len(cat_labels_dict_text)
#cat_labels_dict_text

In [169]:
#select the labels for the relevant cocoID
sub_label = []
sub_DF

error = 0
subcatlab = [None] * len(sub_DF["COCOID"])

for idx, trial in enumerate(sub_DF["COCOID"]):
    #print(trial,idx)
    value_found = cat_labels_dict_text.get(trial)
    if value_found == None:
        error = error + 1
    else:
        subcatlab[idx] = cat_labels_dict_text[trial]
    
#subcatlab

In [ ]:
#Add the labels to the DF next to the cocoIDs

In [167]:
sub_labels = pd.DataFrame({"LABELS": subcatlab})
subj_DF = pd.merge(sub_DF, sub_labels, left_index=True, right_index=True)
subj_DF

,SUBJECT,SESSION,RUN,TRIAL,73KID,COCOID,LABELS
0,1,1,1,1,46003,412922,[animal]
1,1,1,1,2,61883,474858,"[person, sports, vehicle]"
2,1,1,1,3,829,320696,None
3,1,1,1,4,67574,234676,"[person, sports]"
4,1,1,1,5,16021,301595,"[appliance, furniture]"
...,...,...,...,...,...,...,...
29995,1,40,12,58,13774,32606,"[accessory, person, vehicle]"
29996,1,40,12,59,66768,388123,"[appliance, person]"
29997,1,40,12,60,53168,179070,"[person, vehicle]"
29998,1,40,12,61,1944,13597,None
